# Experiment 02: SMTLink-based Verified ReAct Agent - Status Report

## What's Working ✅

### 1. ACL2 Book Successfully Certified
The file experiment-02-smtlink-react.lisp is fully certified with:
- **FTY types**: `agent-state` (9 integer/boolean fields) and `tool-spec` (5 fields)
- **Core functions**: `access-sufficient-p`, `tool-permitted-p`, `tool-budget-sufficient-p`, `can-invoke-tool-p`, `must-respond-p`, `should-continue-p`, `deduct-tool-cost`, `increment-iteration`, `update-satisfaction`, `mark-done`
- **4 SMTLink theorems** with `:smtlink` hints:
  - `permission-safety-smt`
  - `budget-bounds-smt`
  - `termination-bound-smt`
  - `continue-respond-partition-smt`

### 2. SMTLink Integration Setup
- Created experiment-02-smtlink-react.acl2 with proper ttags (`:ttags :all`) for SMTLink's quicklisp/tshell dependencies
- All functions use integer types compatible with SMTLink (no `integerp` as constraint function, no `ifix` wrappers)

---

## The Problem ❌

**SMTLink theorems are being proved by ACL2's clause processors BEFORE reaching the SMT solver.**

Evidence:
1. Certification completes in ~2.6 seconds (too fast for Z3 invocation)
2. No output like `; SMT solver: python /tmp/py_file/smtlink.xxxxx` appears
3. The py_file directory is empty - no Python files generated
4. The expected files (`react_permission_safety.py`, etc.) don't exist

### Why This Happens
SMTLink is a **sequence of clause processors**:
1. `add-hypo-cp` - adds hypotheses
2. `expand-cp` - expands function definitions  
3. `type-extract-cp` - extracts type info
4. `uninterpreted-fn-cp` - handles uninterpreted functions
5. **`SMT-trusted-cp`** - actually calls Z3 (if needed)

Our theorems are simple enough that ACL2 proves them in the earlier clause processors, so Z3 is never invoked and no Python is generated.

### The SMTLink Examples That Work
Looking at examples.lisp, the `poly-ineq-example` theorem involves polynomial inequalities that ACL2 can't solve alone:


In [ ]:
(implies (and (real/rationalp x) (real/rationalp y)
              (<= (+ (* (/ 9 8) x x) (* y y)) 1)
              (<=  (x^2-y^2 x y) 1))
         (< y (- (* 3 (- x (/ 17 8)) (- x (/ 17 8))) 3)))

This *requires* Z3, so Python is generated.

---

## Options for Next Steps

### Option A: Make Theorems Harder
Add constraints that ACL2 can't prove alone but Z3 can (e.g., nonlinear arithmetic, optimization objectives). This feels artificial.

### Option B: Force Z3 Invocation
Investigate SMTLink hints to bypass ACL2's preprocessing and force Z3 invocation. May not be supported.

### Option C: Extract the Translation Logic
Instead of relying on runtime file generation, we could:
1. Study SMTLink's `z3-py/translator.lisp` to understand the ACL2→Python translation
2. Write our own Python module that mirrors the ACL2 spec directly
3. Use Z3 Python API to encode the same constraints

### Option D: Use SMTLink Differently
Focus on theorems that truly need Z3 (optimization, complex arithmetic), and manually write the runtime enforcement Python for the simpler permission/budget checks.

---

## Recommendation

I suggest **Option C + D**: 
1. Use SMTLink for theorems that genuinely need it (optimization problems)
2. Write a Python module (`z3_react_constraints.py`) that directly encodes our ACL2 spec using z3-py, following the patterns from SMTLink's translator

This gives us:
- **Formal verification** via ACL2 (already working)
- **Runtime enforcement** via Z3 Python (we write it, matching the ACL2 spec)
- **Optimization** capability for continue/respond decisions

What would you like to do?